In [3]:
import os
os.chdir("/content/")

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import json 
import random 


In [ ]:
# ensure 100 answer options for each dialog

xsum = json.load(open('drive/MyDrive/xgen_dataset.json'))['data']
medqa = json.load(open('drive/MyDrive/gen_dataset.json'))['data']

for dialogue in medqa['dialogs']:
    for dialog in dialogue['dialog']:
        while len(dialog['answer_options']) < 100:
            dialog['answer_options'].append(random.randrange(0, len(medqa['answers'])))

with open('drive/MyDrive/gen_dataset_new.json', 'w') as jsonFile:
    jsonFile.write(json.dumps({'data': medqa}, indent=4))

print('medqa finished')

for dialogue in xsum['dialogs']:
    for dialog in dialogue['dialog']:
        while len(dialog['answer_options']) < 100:
            dialog['answer_options'].append(random.randrange(0, len(xsum['answers'])))

with open('drive/MyDrive/xgen_dataset_new.json', 'w') as jsonFile:
    jsonFile.write(json.dumps({'data': xsum}, indent=4))

print('xsum finished')


medqa finished
xsum finished


In [ ]:
xsum = json.load(open('drive/MyDrive/xgen_dataset_new.json'))['data']
medqa = json.load(open('drive/MyDrive/gen_dataset_new.json'))['data']

for dialogue in medqa['dialogs']:
    for dialog in dialogue['dialog']:
        if len(dialog['answer_options']) < 100:
            print('not ok', len(dialog['answer_options']))

for dialogue in xsum['dialogs']:
    for dialog in dialogue['dialog']:
        if len(dialog['answer_options']) < 100:
            print('not ok', len(dialog['answer_options']))

In [6]:
!pip install -U transformers==3.0.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 754.6/754.6 KB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 KB 60.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 76.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.2 MB/s eta 0:00:00
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=35c1de8ef8d31fd0d30e064f96ff2a18f71539286f66243d7b7b4331d563a1ca
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


In [7]:
!python -m nltk.downloader punkt

/usr/lib/python3.8/runpy.py:127: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
import torch
torch.cuda.is_available()

True

In [9]:
!git clone https://github.com/patil-suraj/question_generation.git

Cloning into 'question_generation'...
remote: Enumerating objects: 268, done.
remote: Total 268 (delta 0), reused 0 (delta 0), pack-reused 268
Receiving objects: 100% (268/268), 299.04 KiB | 10.68 MiB/s, done.
Resolving deltas: 100% (140/140), done.


In [10]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [11]:
import json
import h5py
import numpy as np
import string
import re
import nltk
import random
import torch
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from question_generation.pipelines import pipeline


def generate_xsum(xsum, gen_pipeline):
    dataset = {}
    summary_dataset = {}
    dataset['questions'], dataset['answers'], dataset['dialogs'] = [], [], []
    
    # Uncomment if you want to continue previous generation and set summary id to the last summary id generated
    # also set the new range of xsum documents to be parsed
    
    # summary_dataset = json.load(open('drive/MyDrive/xsummary_dataset.json'))
    # dataset = json.load(open('drive/MyDrive/xgen_dataset_new.json'))['data']
    
    summary_id = 0
    
    
    print('Parsing dataset initiated ...')
    for i in range(0, 1500):
        if i % 100 == 0:
            dataset_to_create = {
                'data': dataset
            }

            with open('drive/MyDrive/xgen_dataset.json', 'w') as jsonFile:
                jsonFile.write(json.dumps(dataset_to_create, indent=4))

            with open('drive/MyDrive/xsummary_dataset.json', 'w') as jsonFile:
                jsonFile.write(json.dumps(summary_dataset, indent=4))

        # Generate Dialogues from the standard dataset
        

        pair = xsum['train'][i]

        try:
            generated_qas = gen_pipeline(pair['document'])
        except:
            continue
        
        # Store questions' and answers' last ids
        questions_id_counter = len(dataset['questions'])
        answers_id_counter = len(dataset['answers'])
        
        # Generate Dialogs from dataset
        current_dialog = {
            'summary': '',
            'document': '',
            'dialog': [],
        }

        # Set summary and save it to summaries' dataset
        summary_dataset[str(summary_id)] = pair['summary']
        current_dialog['summary'] = str(summary_id)
        summary_id += 1
        
        # Set document
        current_dialog['document'] = pair['document']
        
        
        
        prev_answer_id_counter = answers_id_counter
        for qa in generated_qas:

            questions_id_counter = len(dataset['questions'])
            answers_id_counter = len(dataset['answers'])

            dataset['questions'].append(qa['question'])
            dataset['answers'].append(qa['answer'])

            # Generate Dialogues from generated question-answers
            
            dialogue = {
                'question': len(dataset['questions']) - 1,
                'answer': len(dataset['answers']) - 1,
                'answer_options': [i for i in range(prev_answer_id_counter, len(dataset['answers']))],
                'gt_index': 0,
            }
            dialogue['gt_index'] = len(dialogue['answer_options']) - 1

            current_dialog['dialog'].append(dialogue)
        
        dataset['dialogs'].append(current_dialog)
        
    print('Ready to save dataset ...')
    dataset_to_create = {
        'data': dataset
    }

    with open('drive/MyDrive/xgen_dataset.json', 'w') as jsonFile:
        jsonFile.write(json.dumps(dataset_to_create, indent=4))

    with open('drive/MyDrive/xsummary_dataset.json', 'w') as jsonFile:
        jsonFile.write(json.dumps(summary_dataset, indent=4))
      

    print('Created generated dataset ...')
    
    
    
def main():
    # Question Driven Answer Summarization Primary Dataset path
    print('Loading dataset ...')
    from datasets import load_dataset
    dataset = load_dataset("xsum")

    # Load Pipeline for QA Generation
    print('Loading pipeline ...')
    qa_gen_pipeline = pipeline(
        'multitask-qa-qg', model="valhalla/t5-base-qa-qg-hl")

    # Generate Dataset with VisDial-like structure
    generate_xsum(dataset, qa_gen_pipeline)


if __name__ == '__main__':
    main()

Loading dataset ...


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71. Subsequent calls will reuse this data.


...

Downloading:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Parsing dataset initiated ...
Ready to save dataset ...
Created generated dataset ...


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
import h5py
import numpy as np
import string
import re
import nltk
import random
import torch
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from question_generation.pipelines import pipeline


def generate_dataset(gen_pipeline: pipeline, data: json) -> None:

    # Initialize Data Dictionary
    dataset = {}
    summary_dataset = {}
    dataKeys = data.keys()
    dataset['questions'], dataset['answers'], dataset['dialogs'] = [], [], []

    summary_id = 0

    print('Parsing dataset initiated ...')
    for key in dataKeys:

        # Store questions' and answers' last ids
        questions_id_counter = len(dataset['questions'])
        answers_id_counter = len(dataset['answers'])

        # Append Question
        dataset['questions'].append(data[key]['question'])

        # Append Answers
        answer_keys = list(data[key]['answers'].keys())

        for answer_key in answer_keys:
            dataset['answers'].append(
                data[key]['answers'][answer_key]['answer_ext_summ'])

        # Generate Dialogs from dataset
        current_dialog = {
            'summary': '',
            'document': '',
            'dialog': [],
        }

        # Set summary and save it to summaries' dataset
        summary_dataset[str(summary_id)] = data[key]['multi_ext_summ']
        current_dialog['summary'] = str(summary_id)
        summary_id += 1

        # Set the first article as the dialog's document
        current_dialog['document'] = data[key]['answers'][answer_keys[0]]['article']

        # Generate Dialogues from the standard dataset
        dialogue = {
            'question': questions_id_counter,
            'answer': answers_id_counter,
            'answer_options': [i for i in range(answers_id_counter, len(dataset['answers']))],
            'gt_index': 0,
        }
        dialogue['gt_index'] = 0

        current_dialog['dialog'].append(dialogue)

        # Generate more questions and answers with transformer pipeline
        for answer_key in answer_keys:
            try:
              # Generate questions and answers for each article in the answers
              generated_qas = gen_pipeline(data[key]['answers'][answer_key]['article'])
            except:
                continue

            # Append Results in the generated dataset and Create new dialogues
            prev_answer_id_counter = answers_id_counter
            for qa in generated_qas:

                questions_id_counter = len(dataset['questions'])
                answers_id_counter = len(dataset['answers'])

                dataset['questions'].append(qa['question'])
                dataset['answers'].append(qa['answer'])

                # Generate Dialogues from generated question-answers
              
                dialogue = {
                    'question': len(dataset['questions']) - 1,
                    'answer': len(dataset['answers']) - 1,
                    'answer_options': [i for i in range(prev_answer_id_counter, len(dataset['answers']))],
                    'gt_index': 0,
                }
                dialogue['gt_index'] = len(dialogue['answer_options']) - 1

                current_dialog['dialog'].append(dialogue)

        dataset['dialogs'].append(current_dialog)

    print('Ready to save dataset ...')
    dataset_to_create = {
        'data': dataset
    }

    with open('drive/MyDrive/gen_dataset.json', 'w') as jsonFile:
        jsonFile.write(json.dumps(dataset_to_create, indent=4))

    with open('drive/MyDrive/summary_dataset.json', 'w') as jsonFile:
        jsonFile.write(json.dumps(summary_dataset, indent=4))
      

    print('Created generated dataset ...')


def main():
    # Question Driven Answer Summarization Primary Dataset path
    mediqa_ans_summ_dataset_path = 'drive/MyDrive/question_driven_answer_summarization_primary_dataset.json'

    # Load Pipeline for QA Generation
    print('Loading pipeline ...')
    qa_gen_pipeline = pipeline(
        'multitask-qa-qg', model="valhalla/t5-base-qa-qg-hl")

    # Generate Dataset with VisDial-like structure
    print('Loading dataset ...')
    jsonData = json.load(open(mediqa_ans_summ_dataset_path))

    generate_dataset(qa_gen_pipeline, jsonData)

    pass


if __name__ == '__main__':
    main()


Loading pipeline ...
Loading dataset ...
Parsing dataset initiated ...
Ready to save dataset ...
Created generated dataset ...
